In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

### Функция для получения нужных страниц форума

In [2]:
def parse_forum(url, starts, headers):
    result = []
    for start in starts:
        request = requests.get(url = url + str(start), headers = headers)
        result.append(bs(request.text, 'html.parser'))
    return result

### Функция для спарсивания нужных кусков страницы  
`del` нужен, чтобы удалить закрепленный опрос в начале каждой страницы, который тоже спарсивается.

In [3]:
def parse_soups(soups, tag, class_, delete = True):
    result = []
    for el in soups:
        content = el.find_all(tag, class_ = class_)
        if delete == True:
            del content[0]
        for elem in content:
            result.append(elem.text.strip())
    return result

###  Парсим все страницы форума  
`starts` содержит номер сообщения, с которого будет начинаться страница, на странице помещается 20 сообщений.  
Чтобы спарсить сообщения, которые появятся потом, нужно будет добавить нужные значения в список

In [4]:
starts = [0,20, 40, 60, 80, 100, 120, 140]
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'cookie': '_ga_W0JDW2V29Y=GS1.1.1661181837.2.1.1661184512.60.0.0; _ga=GA1.2.1025230771.1661155454; _gid=GA1.2.721157209.1661155454; style_cookie=null; phpbb3_alft2_k=; phpbb3_alft2_sid=fa5f9f17fa55d5a1a82d62c62159899a; phpbb3_alft2_u=1'
}

soups = parse_forum('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=', starts, headers)

### Парсим из страниц форума содержание и блок «автор», в котором содержится дата

In [5]:
contents = parse_soups(soups, 'div', 'content')
authors = parse_soups(soups, 'p', 'author')

Проверяем, все ли ок с длиной  
Если она соответствует количеству сообщений на форуме, то все ок

*Да, пока я оформлял проект, там уже набежало больше 160 сообщений, так что чтобы спарсить новые, нужно добавить 160 в starts*

In [6]:
print(len(contents))
print(len(authors))

160
160


### Функция для парсинга конкретных данных по регулярному выражению  
`hard = True` нужен, потому что в сообщениях по июлю много «5.», он втупую несколько раз режет оставшееся.

In [7]:
def get_data(regex, list, group_num = 2, hard = False, hard_regex = '(5\.)(.*)'):
    result = []
    for value in list:
        try:
            reg = re.search(regex, value).group(group_num).strip()
            
            if hard == True:
                for i in range(3):
                    try:
                        reg = re.search(hard_regex, reg).group(2).strip()
                    except:
                        break
                        
                        
            result.append(reg)
        except:
            result.append("NaN")
    return result

### Парсим все нужные нам данные по регулярным выражениям  
  
Через `del` удаляем сообщение модератора, оно нам не нужно.  
В `number` генерируем номера сообщений на форуме, чтобы не потеряться. Не так элегантно, как спарсить номера со страниц, но гораздо проще.

In [8]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
city = get_data('(1\.)(.*?)(2\.)', contents)
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
raw_history = get_data('(4\.)(.*?)(5\.)', contents)
raw_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)

del date[0]
del city[0]
del visa_center[0]
del raw_history[0]
del raw_limit[0]

number = list(range(2, len(date) + 2))


### Категоризуем данные визовой истории  
Хочу категоризовать на две группы:
- По длительности;
- По странам — Германия и не Германия.
#### По длительности

In [10]:
history = []

for text in raw_history:
    if (
        'нет' in text.lower()
        ):
        history.append("Нет")
        
    elif (
        'лет' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        ('полг' not in text.lower() or
        'пол ' not in text.lower()) and
        'год' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        'полг' in text.lower() or
        'пол ' in text.lower()
        ):
        history.append('Полугодовые')
        
    elif (
        "мес" in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'дне' in text.lower() or
        'кратко' in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        history.append('Даты поездки')
        
    elif (
        'чист' in text.lower() or
        'никогда' in text.lower() or
        'не бы' in text.lower()
        ):
        history.append('Нет')
        
    else:
        history.append('_Не пон_' + text)

#### По странам
Тут я сделал достаточно тупо, оставил больше работы на ручную проверку

In [11]:
history_country = []

for text in raw_history:
    
    if (
        'немец' in text.lower() or
        'немц' in text.lower() or
        'герма' in text.lower() and
        'отка' not in text.lower()
        ):
        history_country.append("Германия в истории")
        
    elif (
        'чист' in text.lower() or
        'никогда' in text.lower() or
        'не бы' in text.lower()
        ):
        history_country.append('Нет')
        
    else:
        history_country.append('Другие страны')

### Категоризуем длину полученных виз

In [12]:
limit = []

for id, text in enumerate(raw_limit):
    if (
        'отказ' in text.lower()
        ):
        limit.append("Отказ")
        
    elif (
        'лет' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' not in text.lower() and
        'год' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол ' in text.lower() or
        'полг' in text.lower()
        ):
        limit.append('Полгода')
        
    elif (
        "мес" in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'дне' in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        limit.append('Даты поездки')
        
    else:
        limit.append(str(id) +'_Не пон_' + text)

### Собираем датафрейм

In [13]:
df = pd.DataFrame()

df['number'] = number
df['date'] = date
df['city'] = city
df['visa_center'] = visa_center
df['history'] = history
df['history_country'] = history_country
df['raw_history'] = raw_history
df['limit'] = limit
df['raw_limit'] = raw_limit


df

,number,date,city,visa_center,history,history_country,raw_history,limit,raw_limit
0,2,28 янв 2021,Москва,ВЦ Визаметрик,Годовые и больше,Германия в истории,виза Германии на пол года,Год и больше,1 год
1,3,09 мар 2021,Екатеринбург,ВЦ Визаметрик,Годовые и больше,Германия в истории,Годовые чешская и две немецкие,Год и больше,3 года
2,4,26 мар 2021,Москва,ВЦ Визаметрик,Годовые и больше,Другие страны,"Испания, 1 год",Год и больше,3 года
3,5,19 май 2021,Москва,Визаметрик,Годовые и больше,Германия в истории,"Чехия полгода, Германия год.",Год и больше,1 год
4,6,22 май 2021,Москва,Визаметрик,Нет,Нет,Не было виз,Меньше полугода,3 месяца
...,...,...,...,...,...,...,...,...,...
154,156,31 авг 2022,Москва,ВЦ Visametric,Даты поездки,Другие страны,Новый паспорт с венгерской визой под даты поез...,Год и больше,"1 год (мульт), просили 5"
155,157,31 авг 2022,Москва,ВЦ,Годовые и больше,Другие страны,"В 2019 была двухлетняя латвийская, раньше тоже...",Год и больше,2 года
156,158,31 авг 2022,Москва,ВЦ,Годовые и больше,Нет,"08. подача, 29.08. сообщение о готовности (3 р...",Год и больше,2 года (просила на 3)
157,159,01 сен 2022,СПб,ВЦ,Годовые и больше,Другие страны,"Итальянская 2020 на полгода, греческая 2016 на...",Полгода,Пол года (просила на 3)
